Extract Document Content

In [9]:
# !pip install pymupdf openai tiktoken python-dotenv

In [48]:
import fitz  # PyMuPDF
import re
import os
from pathlib import Path
from typing import List

def extract_pdf_text(pdf_path):
    """Extract clean text from PDF, handles images/tables automatically"""
    doc = fitz.open(pdf_path)
    text = []
    
    for page in doc:
        # Extract text blocks (ignores images, keeps readable content)
        page_text = page.get_text("text")
        text.append(page_text.strip())
    
    doc.close()
    return "\n\n".join(text)

def clean_text(raw_text: str) -> str:
    """Aggressive OCR noise removal + semantic cleanup"""
    
    # Step 1: Remove heavy OCR garbage (symbols, random letters, prices)
    text = re.sub(r'[$€£¥]\d+\.?\d*|\b[kx]+|\b[vj]+|\b[L»]+\b|[•/\\|]+|[%#*®]+|[-]{2,}', ' ', raw_text)
    text = re.sub(r'\b\w{1,2}[^a-zA-Z\s]{1,2}\b', ' ', text)  # Short gibberish
    
    # Step 2: Remove common OCR artifacts (CIVIV/M, OMNIVM, etc.)
    text = re.sub(r'\b[A-Z]{2,}M\b|\bfoMccui\b|\bM®e®\b|\bL[V^]+\b', ' ', text)
    text = re.sub(r'\b\w+wm\)\b|\bzxwm\)\b', ' ', text)
    
    # Step 3: Normalize whitespace aggressively
    text = re.sub(r'\s*\n\s*\n\s*\n+', '\n\n', text)  # Multiple newlines -> double
    text = re.sub(r'[ \t]+', ' ', text)  # Multiple spaces/tabs -> single space
    text = re.sub(r'\n[ \t]*\n', '\n\n', text)  # Clean single newlines
    
    # Step 4: Remove lines with mostly noise (short lines, all caps fragments)
    lines = text.split('\n')
    clean_lines = []
    for line in lines:
        line = line.strip()
        if len(line) > 20 and not re.match(r'^[A-Z\s\W]+$', line):  # Skip short/noisy lines
            clean_lines.append(line)
    
    text = '\n\n'.join(clean_lines)
    
    # Step 5: Final polish - remove excessive spaces, strip
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    
    # Step 6: Remove common publisher fragments
    text = re.sub(r'boston PUBLIC LIBRARY\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'HAWTHORN BOOKS INC?\.?\b.*?(?=Copyright|$)', '', text, flags=re.IGNORECASE)
    
    return text.strip()

def process_folder(folder_path):
    """Process all PDFs in folder, clean text, save as text files in Extracted_text folder"""
    folder = Path(folder_path)
    extracted_folder = folder / "Extracted_text"
    extracted_folder.mkdir(exist_ok=True)  # Create folder if it doesn't exist
    docs = []
    
    for pdf_file in folder.glob("*.pdf"):
        print(f"Processing: {pdf_file.name}")
        
        # Extract raw text
        raw_text = extract_pdf_text(pdf_file)
        
        # Clean the text
        cleaned_text = clean_text(raw_text)
        
        # Save individual text file to Extracted_text folder
        txt_path = extracted_folder / f"{pdf_file.stem}.txt"
        txt_path.write_text(cleaned_text)
        
        docs.append({
            "source": pdf_file.name,
            "text": cleaned_text,  # Store cleaned version
            "chunk_id": f"{pdf_file.stem}_full"
        })
        
        print(f"  -> Extracted {len(raw_text):,} chars -> {len(cleaned_text):,} chars cleaned -> Saved: {txt_path.name}")
    
    return docs


# Usage
pdf_folder = "PDFs"
documents = process_folder(pdf_folder)

Processing: All about repairing small appliances.pdf
  -> Extracted 244,610 chars -> 154,095 chars cleaned -> Saved: All about repairing small appliances.txt
Processing: Care and repair of your large home appliances.pdf
  -> Extracted 138,342 chars -> 100,813 chars cleaned -> Saved: Care and repair of your large home appliances.txt
Processing: All about repairing major household appliances.pdf
  -> Extracted 144,034 chars -> 129,411 chars cleaned -> Saved: All about repairing major household appliances.txt
Processing: All thumbs guide to VCRs.pdf
  -> Extracted 104,793 chars -> 89,572 chars cleaned -> Saved: All thumbs guide to VCRs.txt


In [49]:
documents[0]["text"][:2000]

'Illustrated by Carl Bryant W. Clement Stone, Publisher s nice, of course, to have a handyman around the house, but never there when the steam steam or the toaster won’t is written for and dedicated to the housewife who needs a repair manual on ast array of small electrical appliances. Edited by the Staff of Vocational Horizons, Inc. rights reserved, including the right to reproduce this book or portions thereof in any form, except inquiries should be addressed Madison Avenue, New York, New York of America and published taneously in Canada by Prentice-Hall of Canada, Limited, Birch- mount Road, Scarborough, Ontario. Library of Congress Catalog Card I: Repair Problems and Their Solutions Iron does not slide smoothly over cloth Not enough steam comes from the iron Water and steam sputters out of steam ports during operation Iron does not spray water Iron does not shut off Toaster does not heat One side of bread does not toast Bread does not stay down Toaster does not pop up Bread toasts 

In [46]:
len(documents)

4

In [55]:
documents[0].keys()

dict_keys(['source', 'text', 'chunk_id'])

Create Document Chunk

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
splits = text_splitter.split_documents(eur_lex_docs)

In [61]:
# !pip install pinecone
# !pip install python-dotenv

In [66]:
from dotenv import load_dotenv
import os
from pinecone import Pinecone, ServerlessSpec

# This looks for a .env file in the current directory or parent directories
load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

print("Pinecone key loaded:", bool(PINECONE_API_KEY))

Pinecone key loaded: True


In [81]:
# pc = Pinecone(
#     api_key=os.environ["PINECONE_API_KEY"]
# )

# 2️⃣ Create index if it doesn't exist
if "appliance-care-data" not in pc.list_indexes().names():
    pc.create_index(
        name="appliance-care-data",
        dimension=384,          # must match embedding size
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"   # pick your cloud region
        )
    )

In [82]:
index_info = pc.describe_index("appliance-care-data")
print(index_info)

{'deletion_protection': 'disabled',
 'dimension': 384,
 'host': 'appliance-care-data-z61yr43.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'appliance-care-data',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'},
 'tags': None,
 'vector_type': 'dense'}


In [83]:
from dotenv import load_dotenv
import os

# Load .env file from current or parent directory
load_dotenv()

import openai
from pinecone import Pinecone

# Now env vars are available
openai.api_key = os.getenv("OPENAI_API_KEY")
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index("appliance-care-data")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print("Openai api key loaded:", bool(OPENAI_API_KEY))




Openai api key loaded: True


In [84]:
import openai
from pinecone import Pinecone
import os
from pathlib import Path
# from langchain_pinecone import Pinecone as LangPinecone
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize HuggingFace embeddings (384-dim matches your updated index)
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Initialize Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index("appliance-care-data")

def chunk_text(text, chunk_size=800, overlap=150):
    """Split text into overlapping chunks"""
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        if len(chunk.strip()) > 50:  # Skip tiny chunks
            chunks.append(chunk.strip())
        start = max(start + chunk_size - overlap, end)
    return chunks

def embed_and_upsert(documents, batch_size=100):
    """Chunk, embed with HuggingFace, upsert to Pinecone"""
    all_vectors = []
    
    for doc in documents:
        # Chunk the text
        chunks = chunk_text(doc["text"])
        
        for i, chunk in enumerate(chunks):
            # Generate embedding with HuggingFace
            embedding = embedding_model.embed_query(chunk)  # Returns 384-dim vector
            
            # Prepare Pinecone record
            vector = {
                "id": f"{doc['chunk_id']}_{i+1}",
                "values": embedding,
                "metadata": {
                    "source": doc["source"],
                    "text": chunk[:1000],  # Truncate for metadata limits
                    "chunk_index": i+1,
                    "total_chunks": len(chunks)
                }
            }
            all_vectors.append(vector)
    
    # Upsert in batches
    for i in range(0, len(all_vectors), batch_size):
        batch = all_vectors[i:i+batch_size]
        index.upsert(vectors=batch)
        print(f"Upserted batch {i//batch_size + 1} ({len(batch)} vectors)")
    
    print(f"✅ Populated {len(all_vectors)} vectors to Pinecone (384-dim)")

# Run it
embed_and_upsert(documents)

/opt/homebrew/lib/python3.11/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore
/var/folders/kt/3_xkvbt50nl1g9rk19fkhzq00000gn/T/ipykernel_55396/4149373840.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should

Upserted batch 1 (100 vectors)
Upserted batch 2 (100 vectors)
Upserted batch 3 (100 vectors)
Upserted batch 4 (100 vectors)
Upserted batch 5 (100 vectors)
Upserted batch 6 (93 vectors)
✅ Populated 593 vectors to Pinecone (384-dim)


In [87]:
index_name="appliance-care-data"
dense_index = pc.Index(index_name)


# View stats for the index
stats = dense_index.describe_index_stats()
print(stats)

{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 593}},
 'total_vector_count': 593,
 'vector_type': 'dense'}


Test Basic Search functionality

In [89]:
def query_pinecone(query_text, top_k=5):
    # Embed the query
    query_embedding = embedding_model.embed_query(query_text)
    
    # Search Pinecone
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True  # Returns text/source info
    )
    
    # Format results
    for match in results['matches']:
        score = match['score']
        text = match['metadata']['text']
        source = match['metadata']['source']
        print(f"Score: {score:.3f} | Source: {source}")
        print(f"Text: {text[:200]}...\n")
    
    return results['matches']

# Example usage
query_pinecone("Iron burns cloth, what to do?")


Score: 0.624 | Source: All about repairing small appliances.pdf
Text: with the end of a paper caution: To avoid damaging the sole plate of the iron, never allow sharp objects or hard, rough surfaces in contact with the sole plate. Always rest and Clothes Irons (Steam On...

Score: 0.623 | Source: All about repairing small appliances.pdf
Text: s or those involving any possible risk clearly must be referred to the attention of a qualified repair technician. In many instances the author suggests tools and materials to accomplish the adjustmen...

Score: 0.586 | Source: All about repairing small appliances.pdf
Text: ments wise turns at a time) until iron steams properly. is defective and should be replaced by a qualified appliance technician. For final test, allow iron to cool to room tempera- ture. Then set fabr...

Score: 0.538 | Source: All about repairing small appliances.pdf
Text: ay mechanism has a serious mechanical defect, and the iron should he taken to a qualified appliance the d

[{'id': 'All about repairing small appliances_full_8',
  'metadata': {'chunk_index': 8.0,
               'source': 'All about repairing small appliances.pdf',
               'text': 'with the end of a paper caution: To avoid damaging the '
                       'sole plate of the iron, never allow sharp objects or '
                       'hard, rough surfaces in contact with the sole plate. '
                       'Always rest and Clothes Irons (Steam Only) Not enough '
                       'steam comes from the iron. Explanation: Minerals the '
                       'steam chamber and steam ports and can clog the Tools '
                       'and Materials Needed: a paper clip or hairpin I 2 cup '
                       'of white inegar with a paper clip or a hairpin bent '
                       'into shape as ( Figure 1) Wiggle paper clip or hair- '
                       'pin in the steam ports to loosen hardened mineral white '
                       'inegar with 2 cup of 